In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from tqdm import tqdm, trange

device = 'cuda:0'

In [2]:
k = 8
N = 16

nb_epoch = 2**18
batch_size = 256

train_SNR_Es = 10
test_SNR_Es = np.arange(0,11)

In [3]:
class FCNNDecoder(torch.nn.Module):
    def __init__(self, code_n, code_k):
        super(FCNNDecoder, self).__init__()
        
        self.dec_lays = torch.nn.Sequential(
            torch.nn.Linear(in_features=code_n, out_features=128),
            torch.nn.Linear(in_features=128, out_features=64),
            torch.nn.Linear(in_features=64, out_features=32),
            torch.nn.Linear(in_features=32, out_features=code_k)
        )


    def forward(self, inputs):
        logits = self.dec_lays(inputs)
        return logits

In [4]:
class NoisyDataset(torch.utils.data.Dataset):
    def __init__(self, codewords, infwords, SNR_db_Es, active_cw_indices):        
        sigma = torch.sqrt(1/(2*10**(torch.tensor(SNR_db_Es)/10)))
        bpsk_sig = -2*codewords[active_cw_indices,:] + 1
        noisy_sig = torch.randn_like(bpsk_sig) * sigma + bpsk_sig
        self.llrs = 2 * noisy_sig / sigma**2
        self.active_infwords = infwords[active_cw_indices,:]
            
    def __len__(self):
        return self.llrs.shape[0]
    
    def __getitem__(self, idx):
        return self.llrs[idx], self.active_infwords[idx]

In [5]:
def half_adder(a,b):
    s = a ^ b
    c = a & b
    return s,c

def full_adder(a,b,c):
    s = (a ^ b) ^ c
    c = (a & b) | (c & (a ^ b))
    return s,c

def add_bool(a,b):
    if len(a) != len(b):
        raise ValueError('arrays with different length')
    k = len(a)
    s = np.zeros(k,dtype=bool)
    c = False
    for i in reversed(range(0,k)):
        s[i], c = full_adder(a[i],b[i],c)    
    if c:
        warnings.warn("Addition overflow!")
    return s

def inc_bool(a):
    k = len(a)
    increment = np.hstack((np.zeros(k-1,dtype=bool), np.ones(1,dtype=bool)))
    a = add_bool(a,increment)
    return a

def bitrevorder(x):
    m = np.amax(x)
    n = np.ceil(np.log2(m)).astype(int)
    for i in range(0,len(x)):
        x[i] = int('{:0{n}b}'.format(x[i],n=n)[::-1],2)  
    return x

def int2bin(x,N):
    if isinstance(x, list) or isinstance(x, np.ndarray):
        binary = np.zeros((len(x),N),dtype='bool')
        for i in range(0,len(x)):
            binary[i] = np.array([int(j) for j in bin(x[i])[2:].zfill(N)])
    else:
        binary = np.array([int(j) for j in bin(x)[2:].zfill(N)],dtype=bool)
    
    return binary

def bin2int(b):
    if isinstance(b[0], list):
        integer = np.zeros((len(b),),dtype=int)
        for i in range(0,len(b)):
            out = 0
            for bit in b[i]:
                out = (out << 1) | bit
            integer[i] = out
    elif isinstance(b, np.ndarray):
        if len(b.shape) == 1:
            out = 0
            for bit in b:
                out = (out << 1) | bit
            integer = out     
        else:
            integer = np.zeros((b.shape[0],),dtype=int)
            for i in range(0,b.shape[0]):
                out = 0
                for bit in b[i]:
                    out = (out << 1) | bit
                integer[i] = out
        
    return integer

def polar_design_awgn(N, k, design_snr_dB):  
        
    S = 10**(design_snr_dB/10)
    z0 = np.zeros(N)

    z0[0] = np.exp(-S)
    for j in range(1,int(np.log2(N))+1):
        u = 2**j
        for t in range(0,int(u/2)):
            T = z0[t]
            z0[t] = 2*T - T**2     # upper channel
            z0[int(u/2)+t] = T**2  # lower channel
        
    # sort into increasing order
    idx = np.argsort(z0)
        
    # select k best channels
    idx = np.sort(bitrevorder(idx[0:k]))
    
    A = np.zeros(N, dtype=bool)
    A[idx] = True
        
    return A

def polar_transform_iter(u):

    N = len(u)
    n = 1
    x = np.copy(u)
    stages = np.log2(N).astype(int)
    for s in range(0,stages):
        i = 0
        while i < N:
            for j in range(0,n):
                idx = i+j
                x[idx] = x[idx] ^ x[idx+n]
            i=i+2*n
        n=2*n
    return x

In [6]:
def create_words(code):
    # Create all possible information words
    d = np.zeros((2**k,k),dtype=bool)
    for i in range(1,2**k):
        d[i]= inc_bool(d[i-1])

    # Create sets of all possible codewords (codebook)
    if code == 'polar':   

        A = polar_design_awgn(N, k, design_snr_dB=0)  # logical vector indicating the nonfrozen bit locations 
        x = np.zeros((2**k, N),dtype=bool)
        u = np.zeros((2**k, N),dtype=bool)
        u[:,A] = d

        for i in range(0,2**k):
            x[i] = polar_transform_iter(u[i])
        return x, d, A

    elif code == 'random':

        np.random.seed(4267)   # for a 16bit Random Code (r=0.5) with Hamming distance >= 2
        x = np.random.randint(0,2,size=(2**k,N), dtype=bool)
        return x, d

In [7]:
code = 'polar'              # type of code ('random' or 'polar')
codewords, inputs, fr_pos = create_words(code)
codewords = torch.tensor(codewords, dtype=torch.float32).to(device)
inputs = torch.tensor(inputs, dtype=torch.float32).to(device)

In [8]:
def train(model, codewords, inputs, n_epochs, train_indices):
    
    train_batch_size = train_indices.shape[0]

    loss_fn = torch.nn.BCEWithLogitsLoss()
    opt = torch.optim.Adam(model.parameters(),lr=1e-3)

    t = trange(n_epochs, desc='', leave=False)
    for epoch in t:
        train_dataset = NoisyDataset(codewords=codewords, infwords=inputs, 
                                    SNR_db_Es=train_SNR_Es, active_cw_indices=train_indices)
        train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=train_batch_size, shuffle=True)

        train_ep(model, train_loader, loss_fn, opt)

def train_ep(model, train_loader, loss_fn, opt):
    model.train()
    for llrs, iws in train_loader:
        logits = model(llrs)
        bce_loss = loss_fn(-1*logits, iws)
        opt.zero_grad()
        bce_loss.backward()
        opt.step()

def evaluate(model, codewords, infwords, test_indices, snr_range, max_iter=1e4):
    model.eval()

    nb_bit_errors = np.zeros_like(snr_range)
    nb_frame_errors = np.zeros_like(snr_range)
    nb_bits = np.zeros_like(snr_range)
    nb_frames = np.zeros_like(snr_range)

    with torch.no_grad():
        t = tqdm(snr_range, desc='', leave=False)
        for i, snr in enumerate(t):
            iter_counter = 0
            while iter_counter < max_iter:
                iter_counter += 1
                test_dataset = NoisyDataset(codewords=codewords, infwords=infwords, 
                                            SNR_db_Es=snr, active_cw_indices=test_indices)
                loader = torch.utils.data.DataLoader(test_dataset, batch_size=test_indices.shape[0])
                for llrs, iw in loader:
                    logits = model(llrs)
                    err_vec = torch.fmod((iw + (logits < 0)), 2)
                    nb_bit_errors[i] += torch.sum(err_vec)
                    nb_frame_errors[i] += torch.sum(torch.sum(err_vec, dim=1) > 0)

                nb_bits[i] += (len(loader.dataset) * infwords.shape[1])
                nb_frames[i] += len(loader.dataset)

            t.set_description(f'EsN0: {snr:.2f} dB', refresh=True)

    return nb_bit_errors/nb_bits, nb_frame_errors/nb_frames



# Train on first 16 codewords

In [9]:
train_indices = np.arange(16)
test_indices = np.arange(16,2**k)

model = FCNNDecoder(code_n=N, code_k=k).to(device)

In [ ]:
train(model=model,codewords=codewords,inputs=inputs,n_epochs=nb_epoch,train_indices=train_indices)

In [12]:
evaluate(model=model, codewords=codewords, infwords=inputs, test_indices=test_indices, snr_range=test_SNR_Es, max_iter=1e2)

(array([0.50693229, 0.50694271, 0.50774479, 0.50908333, 0.50911458,
        0.50863021, 0.50832292]),
 array([0.99575   , 0.99666667, 0.99658333, 0.99641667, 0.99704167,
        0.99695833, 0.997125  ]))

In [ ]:
def evaluate_per_cw(model, codewords, infwords, test_indices, snr_range, max_iter=1e4):
    model.eval()

    nb_bit_errors = torch.zeros(size=(snr_range.shape[0],test_indices.shape[0]))
    nb_frame_errors = torch.zeros(size=(snr_range.shape[0],test_indices.shape[0]))
    nb_bits = torch.zeros_like(torch.tensor(snr_range))
    nb_frames = torch.zeros_like(torch.tensor(snr_range))

    with torch.no_grad():
        t = tqdm(snr_range, desc='', leave=False)
        for i, snr in enumerate(t):
            iter_counter = 0
            while iter_counter < max_iter:
                iter_counter += 1
                test_dataset = NoisyDataset(codewords=codewords, infwords=infwords, 
                                            SNR_db_Es=snr, active_cw_indices=test_indices)
                loader = torch.utils.data.DataLoader(test_dataset, batch_size=test_indices.shape[0])
                for llrs, iw in loader:
                    logits = model(llrs)
                    err_vec = torch.fmod((iw + (logits < 0)), 2)
                    nb_bit_errors[i,:] += torch.sum(err_vec,dim=1)
                    nb_frame_errors[i,:] += (torch.sum(err_vec, dim=1) > 0)

                # nb_bits[i] += infwords.shape[1]
                # nb_frames[i] += 1

            t.set_description(f'EsN0: {snr:.2f} dB', refresh=True)

    # print(nb_bit_errors.shape, nb_bits.repeat(1,test_indices.shape[0]).shape, nb_frame_errors.shape, nb_frames.repeat(1,test_indices.shape[0]).shape)
    return nb_bit_errors/max_iter/infwords.shape[1], nb_frame_errors/max_iter

In [ ]:
ber_out, fer_out = evaluate_per_cw(model=model, codewords=codewords, infwords=inputs, test_indices=np.arange(2**k), snr_range=test_SNR_Es, max_iter=1e4)

# Train on random 16 codewords

In [9]:
train_size = 16
np.random.seed(seed=1337)
train_indices = np.random.choice(2**k, size=train_size, replace=False)
model = FCNNDecoder(code_n=N, code_k=k).to(device)

test_indices = np.arange(2**k)

In [10]:
train(model=model,codewords=codewords,inputs=inputs,n_epochs=nb_epoch,train_indices=train_indices)

In [11]:
ber_out_16, fer_out_16 = evaluate(model=model, codewords=codewords, infwords=inputs, test_indices=test_indices, snr_range=test_SNR_Es, max_iter=1e3)

# Train on random 32 codewords

In [12]:
train_size = 32
np.random.seed(seed=1337)
train_indices = np.random.choice(2**k, size=train_size, replace=False)
model = FCNNDecoder(code_n=N, code_k=k).to(device)

test_indices = np.arange(2**k)

In [13]:
train(model=model,codewords=codewords,inputs=inputs,n_epochs=nb_epoch,train_indices=train_indices)

In [14]:
ber_out_32, fer_out_32 = evaluate(model=model, codewords=codewords, infwords=inputs, test_indices=test_indices, snr_range=test_SNR_Es, max_iter=1e3)

# Train on random 48 codewords

In [15]:
train_size = 48
np.random.seed(seed=1337)
train_indices = np.random.choice(2**k, size=train_size, replace=False)
model = FCNNDecoder(code_n=N, code_k=k).to(device)

test_indices = np.arange(2**k)

In [16]:
train(model=model,codewords=codewords,inputs=inputs,n_epochs=nb_epoch,train_indices=train_indices)

  5%|▍         | 12150/262144 [00:32<13:30, 308.49it/s]

In [ ]:
ber_out_48, fer_out_48 = evaluate(model=model, codewords=codewords, infwords=inputs, test_indices=test_indices, snr_range=test_SNR_Es, max_iter=1e3)

# Train on random 64 codewords

In [ ]:
train_size = 64
np.random.seed(seed=1337)
train_indices = np.random.choice(2**k, size=train_size, replace=False)
model = FCNNDecoder(code_n=N, code_k=k).to(device)

test_indices = np.arange(2**k)

In [ ]:
train(model=model,codewords=codewords,inputs=inputs,n_epochs=nb_epoch,train_indices=train_indices)

In [ ]:
ber_out_64, fer_out_64 = evaluate(model=model, codewords=codewords, infwords=inputs, test_indices=test_indices, snr_range=test_SNR_Es, max_iter=1e3)

# Train on random 86 codewords

In [ ]:
train_size = 86
np.random.seed(seed=1337)
train_indices = np.random.choice(2**k, size=train_size, replace=False)
model = FCNNDecoder(code_n=N, code_k=k).to(device)

test_indices = np.arange(2**k)

In [ ]:
train(model=model,codewords=codewords,inputs=inputs,n_epochs=nb_epoch,train_indices=train_indices)

In [ ]:
ber_out_86, fer_out_86 = evaluate(model=model, codewords=codewords, infwords=inputs, test_indices=test_indices, snr_range=test_SNR_Es, max_iter=1e3)

# Train on random 128 codewords

In [ ]:
train_size = 128
np.random.seed(seed=1337)
train_indices = np.random.choice(2**k, size=train_size, replace=False)
model = FCNNDecoder(code_n=N, code_k=k).to(device)

test_indices = np.arange(2**k)

In [ ]:
train(model=model,codewords=codewords,inputs=inputs,n_epochs=nb_epoch,train_indices=train_indices)

In [ ]:
ber_out_128, fer_out_128 = evaluate(model=model, codewords=codewords, infwords=inputs, test_indices=test_indices, snr_range=test_SNR_Es, max_iter=1e3)

# Train on all codewords

In [ ]:
train_size = 256
np.random.seed(seed=1337)
train_indices = np.random.choice(2**k, size=train_size, replace=False)
model = FCNNDecoder(code_n=N, code_k=k).to(device)

test_indices = np.arange(2**k)

In [ ]:
train(model=model,codewords=codewords,inputs=inputs,n_epochs=nb_epoch,train_indices=train_indices)

In [ ]:
ber_out_256, fer_out_256 = evaluate(model=model, codewords=codewords, infwords=inputs, test_indices=test_indices, snr_range=test_SNR_Es, max_iter=1e3)